# Preprocess

In [3]:
''' Handle the necessary imports'''
import cv2
import numpy as np
import os
import glob

Now, after importing libraries, we load and check if the images are properly displayed.

In [9]:
aflw_image = []
for img in glob.glob("Data//AFLW//positive_faces//*.png"):
    n= cv2.imread(img)
    aflw_image.append(n)
    
print(len(aflw_image))

cv2.imshow("Image",aflw_image[1])
cv2.waitKey(500)
cv2.destroyAllWindows()

"""
for index in range(1):
    loaded_image_aflw=cv2.imread('Data//AFLW//positive_faces//positive_face_'+str(index)+'.png')  # load the image from disk for AFLW dataset
    loaded_image_lfw=cv2.imread('Data//LFW//positive_faces//positive_face_'+str(index)+'.png')  # load the image from disk for LFW dataset
    cv2.imshow("Loaded Image",loaded_image_aflw) # show image
    cv2.waitKey(5000)
cv2.destroyAllWindows()
"""

2493


'\nfor index in range(1):\n    loaded_image_aflw=cv2.imread(\'Data//AFLW//positive_faces//positive_face_\'+str(index)+\'.png\')  # load the image from disk for AFLW dataset\n    loaded_image_lfw=cv2.imread(\'Data//LFW//positive_faces//positive_face_\'+str(index)+\'.png\')  # load the image from disk for LFW dataset\n    cv2.imshow("Loaded Image",loaded_image_aflw) # show image\n    cv2.waitKey(5000)\ncv2.destroyAllWindows()\n'

Now that we have tested load and display of the images,we can proceed with resizing and the training and test data split.

In [ ]:
patch_size=60 #60*60*3
num_training_images=1000
num_testing_images=100

In [13]:
index=0
while index<num_training_images:
    loaded_image=cv2.imread('Data//AFLW//positive_faces//positive_face_'+str(index)+'.png')  # load the image from disk 
    resized_image=cv2.resize(loaded_image,(patch_size,patch_size),cv2.INTER_LANCZOS4) #resize it
    cv2.imwrite('Data//AFLW//extracted_faces//train//train_face'+str(index)+'.png',resized_image)  # save the image to disk 
    #repeat for non-faces 
    #loaded_image=cv2.imread('Data//WIDER_train//images//0--Parade'+str(index)+'.png')  # load the image from disk 
    #resized_image=cv2.resize(loaded_image,(patch_size,patch_size),cv2.INTER_LANCZOS4) #resize it
    #cv2.imwrite('Data//AFLW//extracted_faces//train_face'+str(index)+'.png',resized_image)  # save the image to disk
    index+=1 #increment
    #cv2.imshow("Loaded Image",resized_image) # show image
    #cv2.waitKey(1000)

Similarly process for testing images.

In [15]:
index=1000
while index<num_testing_images+num_training_images:
    loaded_image=cv2.imread('Data//AFLW//positive_faces//positive_face_'+str(index)+'.png')  # load the image from disk 
    resized_image=cv2.resize(loaded_image,(patch_size,patch_size),cv2.INTER_LANCZOS4) #resize it
    cv2.imwrite('Data//AFLW//extracted_faces//test//test_face'+str(index-num_training_images)+'.png',resized_image)  # save the image to disk 
    #repeat for non-faces 
    #loaded_image=cv2.imread('Data//WIDER_train//images//0--Parade'+str(index)+'.png')  # load the image from disk 
    #resized_image=cv2.resize(loaded_image,(patch_size,patch_size),cv2.INTER_LANCZOS4) #resize it
    #cv2.imwrite('Data//AFLW//extracted_faces//train_face'+str(index)+'.png',resized_image)  # save the image to disk
    index+=1 #increment
    #cv2.imshow("Loaded Image",resized_image) # show image
    #cv2.waitKey(1000)

Now we can proceed with training.


## PREPARE TRAINING DATA

Model 1. Learn single Gaussian model using training images and report your results as stated
above.

For a single gaussian model, we have the equation as follows.

(1000, 10800)
(1000, 10800)


In [ ]:
# single gaussian model
# Convention, Y-outputs/labels and X-inputs/features
def GMM_train(X,max_iterations=100):
    


In [ ]:

        
def demo_2d():
    
    # Load data
    #X = np.genfromtxt('data1.csv', delimiter=',')
    ### generate the random data     
    np.random.seed(3)
    m1, cov1 = [9, 8], [[.5, 1], [.25, 1]] ## first gaussian
    data1 = np.random.multivariate_normal(m1, cov1, 90)
    
    m2, cov2 = [6, 13], [[.5, -.5], [-.5, .1]] ## second gaussian
    data2 = np.random.multivariate_normal(m2, cov2, 45)
    
    m3, cov3 = [4, 7], [[0.25, 0.5], [-0.1, 0.5]] ## third gaussian
    data3 = np.random.multivariate_normal(m3, cov3, 65)
    X = np.vstack((data1,np.vstack((data2,data3))))
    np.random.shuffle(X)
#    np.savetxt('sample.csv', X, fmt = "%.4f",  delimiter = ",")
    ####
    gmm = GMM(3, 0.000001)
    params = gmm.fit_EM(X, max_iters= 100)
    print (params.log_likelihoods)
    import pylab as plt    
    from matplotlib.patches import Ellipse
    
    def plot_ellipse(pos, cov, nstd=2, ax=None, **kwargs):
        def eigsorted(cov):
            vals, vecs = np.linalg.eigh(cov)
            order = vals.argsort()[::-1]
            return vals[order], vecs[:,order]
    
        if ax is None:
            ax = plt.gca()
    
        vals, vecs = eigsorted(cov)
        theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    
        # Width and height are "full" widths, not radius
        width, height = 2 * nstd * np.sqrt(abs(vals))
        ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwargs)
    
        ax.add_artist(ellip)
        return ellip    
    
    def show(X, mu, cov):

        plt.cla()
        K = len(mu) # number of clusters
        colors = ['b', 'k', 'g', 'c', 'm', 'y', 'r']
        plt.plot(X.T[0], X.T[1], 'm*')
        for k in range(K):
          plot_ellipse(mu[k], cov[k],  alpha=0.6, color = colors[k % len(colors)])  

    
    fig = plt.figure(figsize = (13, 6))
    fig.add_subplot(121)
    show(X, params.mu, params.Sigma)
    fig.add_subplot(122)
    plt.plot(np.array(params.log_likelihoods))
    plt.title('Log Likelihood vs iteration plot')
    plt.xlabel('Iterations')
    plt.ylabel('log likelihood')
    plt.show()
    print (gmm.predict(np.array([1, 2])))
       

if __name__ == "__main__":

    demo_2d()    
    from optparse import OptionParser

    parser = OptionParser()
    parser.add_option("-f", "--file", dest="filepath", help="File path for data")
    parser.add_option("-k", "--clusters", dest="clusters", help="No. of gaussians")    
    parser.add_option("-e", "--eps", dest="epsilon", help="Epsilon to stop")    
    parser.add_option("-m", "--maxiters", dest="max_iters", help="Maximum no. of iteration")        
    options, args = parser.parse_args()
    
    if not options.filepath : raise('File not provided')
    
    if not options.clusters :
        print("Used default number of clusters = 3" )
        k = 3
    else: k = int(options.clusters)
    
    if not options.epsilon :
        print("Used default eps = 0.0001" )
        eps = 0.0001
    else: eps = float(options.epsilon)
    
    if not options.max_iters :
        print("Used default maxiters = 1000" )
        max_iters = 1000
    else: eps = int(options.maxiters)
    
    #X = np.genfromtxt(options.filepath, delimiter=',')
    gmm = GMM(k, eps)
    X=training_data_face(0,0:100)
    params = gmm.fit_EM(X, max_iters)
    print (params.log_likelihoods)
    gmm.plot_log_likelihood()
    print (gmm.predict(np.array([1, 2])))
